In [1]:
import networkx as nx
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
from multiprocessing import Pool, cpu_count
from tqdm.notebook import trange, tqdm
from time import time
from ipywidgets import interact, fixed

import seaborn as sns
sns.set()

import os, sys
currentdir = os.path.dirname(os.path.realpath(__name__))
scriptsdir = os.path.dirname(currentdir)+'/scripts'
sys.path.append(scriptsdir)

from distances import dist_1d, dist_2d_lattice, dist_2d_spacial
from random_walks import simple_2d_random_walk, lazy_2d_random_walk
from generate_graph import Kleinberg_2d, small_world_2d, small_world_2d_new, sparse_lattice_graph

Two necessary functions

In [2]:
def get_path_forG(G, L, max_steps=int(1e2), max_walks=int(1e3),  start_node=(0,0), use_random_starting_node=False,
             random_walk_function=lazy_2d_random_walk):
    '''
    Given a Graph G and a type of random walk, it returns a list containing the nodes visited
    for a random path of time max_steps for each max_walks random path
    
    Arguments:
    - G: graph
    - L: number of nodes on line
    - r: parameter for the long range decay
    - max_steps: time of the random path
    - max_walks: number of random path (walk) considered
    - start_node: initial node
    - use_random_starting_node: if True the starting node is random
    - random_walk_function: simple_2d_random_walk_parallel for default
    
    Return:
    - np.array(paths):
        - paths[i]: list of visited nodes for the i-th walk
        - paths[i][-1]: last node visited by the i-th path
    '''
    # seed list
    seed_list = np.random.permutation(np.arange(0, max_walks)).tolist()

    # list of random walks (dim = (max_walks, max_steps))
    pool = Pool(cpu_count())
    paths = np.array(pool.starmap_async(random_walk_function, [(G, L, max_steps, start_node, use_random_starting_node, seed) for seed in seed_list]).get())[:,:,1]
    pool.close()
    #starting_paths.append(paths)
    
    return paths

def increase_walks_forG(G, L, paths, max_steps=int(1e2), max_walks=int(1e3), start_node=(0,0), use_random_starting_node=False,
             random_walk_function=lazy_2d_random_walk):
    '''
    This increases the walks of the paths got from the get_path_forG
    
    Arguments:
    - G: graph
    - L: number of nodes on line
    - r: parameter for the long range decay
    - max_steps: time of the random path
    - max_walks: number of random path (walk) considered
    - start_node: initial node
    - use_random_starting_node: if True the starting node is random
    - random_walk_function: simple_2d_random_walk_parallel for default
    
    Return:
    - np.array(paths):
        - paths[i]: list of visited nodes for the i-th walk
    '''
    # seed list
    seed_list = np.random.permutation(np.arange(0, max_walks)).tolist()

    # list of random walks (dim = (max_walks, max_steps))
    pool = Pool(cpu_count())
    new_paths = np.array(pool.starmap_async(random_walk_function, 
                        [(G, L, max_steps, start_node, use_random_starting_node, seed) 
                         for seed in seed_list]).get())[:,:,1]
    pool.close()
    increased_paths=np.append(paths,new_paths,axis=0)
    return np.array(increased_paths)

We start by generating some paths with starting node (0,0)

In [3]:
L=20
r=2
G=small_world_2d_new(L,r)
max_steps=100
max_walks=20000
tic=time()
paths=get_path_forG(G,L,max_steps=max_steps, max_walks=max_walks)
toc=time()
print(toc-tic)
print(paths.shape)

12.405115842819214
(20000, 100, 2)


Now the main idea to get new paths with a define starting node called (find_node) is to perform a for cycle and for each path we compute the minimum index where we get the find_node, from it we'll define a new_path and of course it'll have a smaller number of steps, therefore we'll add new steps id order to get the same max_steps.
At the end, since not every path contains the find_node we will add also new path id order to have the same statistic that we can infer from max_walks

In [5]:
def get_new_start(paths,max_steps, max_walks, find_node):
    """
    INPUT:
        paths = is the initial paths
        max_steps = is the fixed number of steps that we want to
        max_walks = is the fixed number of paths that we want to
        find_node = is the starting node that we are looking for
    """
    new_paths=[]
    for path in paths:
        # finding the node
        idx=np.argmax((path==find_node).sum(axis=1))
        """
        idx==0 could also means that we didn't find the find_node in the path, therefore to exclude this possibility
        if the first node reached is not our find_node this means that idx=0 tells us there's no find_node in the path
        """
        if idx==0 :
            if (path[0]!=np.array(find_node)).any():
                # this means that I didn't find the node, hence I continue
                continue
        new_path=path[idx:] # new path
        
        # now we add new steps
        if len(new_path)!=max_steps :
            new_steps=max_steps-len(new_path)
            # new path with starting node (new_path[-1][0],new_path[-1][1]))
            new_path_steps=np.array(lazy_2d_random_walk(G,L,max_steps=new_steps,
                                                    start_node=(new_path[-1][0],new_path[-1][1])))[:,1]
            # here we merge the two paths
            new_path=np.append(new_path,new_path_steps,axis=0)
        new_paths.append(new_path)
    new_paths=np.array(new_paths)
    
    # here we add new walks, precisely max_walks-new_paths.shape[0]
    new_paths=increase_walks_forG(G,L,new_paths,max_steps=max_steps, 
                              max_walks=max_walks-new_paths.shape[0],start_node=find_node)
    return new_paths

Let's try to find new paths for find_node=(4,2)

In [6]:
find_node=(4,2)
tic=time()
new_paths=get_new_start(paths, max_steps, max_walks, find_node)
toc=time()
print(toc-tic)
print(new_paths.shape)

7.840587139129639
(20000, 100, 2)


Or for find_node=(13,17)

In [7]:
find_node=(13,17)
tic=time()
new_paths=get_new_start(paths, max_steps, max_walks, find_node)
toc=time()
print(toc-tic)
print(new_paths.shape)

8.748890161514282
(20000, 100, 2)


What if I would had to compute the entire paths?

In [8]:
start_node=(4,2)
tic=time()
paths=get_path_forG(G,L,max_steps=max_steps, max_walks=max_walks, start_node=start_node)
toc=time()
print(toc-tic)
print(paths.shape)

12.7581148147583
(20000, 100, 2)


In [9]:
start_node=(13,17)
tic=time()
paths=get_path_forG(G,L,max_steps=max_steps, max_walks=max_walks, start_node=start_node)
toc=time()
print(toc-tic)
print(paths.shape)

13.602208852767944
(20000, 100, 2)
